In [14]:
# import and paths 

import os
import pickle
import numpy as np
import pandas as pd

PROJECT_ROOT = os.path.abspath("..")
DATA_PROCESSED = os.path.join(PROJECT_ROOT, "data", "processed")


In [15]:
# Load risk and regime data 

with open(os.path.join(DATA_PROCESSED, "covariance_matrices.pkl"), "rb") as f:
    scenario_covs = pickle.load(f)

market_regimes = pd.read_csv(
    os.path.join(DATA_PROCESSED, "market_regimes.csv"),
    index_col=0,
    parse_dates=True
)

dates = list(scenario_covs.keys())
len(dates)


41

In [16]:
# define binary decision variables 

# Number of assets inferred from covariance shape
N_ASSETS = scenario_covs[dates[0]]["base"].shape[0]
N_ASSETS


25

In [17]:
# Previous portfolio state 

# Start from equal-weight portfolio (Day 1)
prev_selection = np.ones(N_ASSETS)
prev_selection



array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1.])

In [18]:
# Hyperparameters (penalty weights)

PARAMS = {
    "lambda_risk": 1.0,
    "lambda_cost": 0.1,
    "lambda_turnover": 0.1,
    "lambda_cardinality": 5.0,
    "cardinality_max": 10
}

PARAMS


{'lambda_risk': 1.0,
 'lambda_cost': 0.1,
 'lambda_turnover': 0.1,
 'lambda_cardinality': 5.0,
 'cardinality_max': 10}

In [19]:
# risk term Q matrix

def risk_term(cov):
    return cov.values


In [20]:
# Linear cost and turnover term Q matrix

def linear_change_term(prev_sel):
    return (1 - 2 * prev_sel)


In [21]:
# cardinality constraint Q matrix

def cardinality_penalty(n, K):
    Q = np.ones((n, n))
    linear = -2 * K * np.ones(n)
    return Q, linear


In [22]:
# Build full QUBO matrix (core functions)

def build_qubo(date, prev_sel, params):
    regime = market_regimes.loc[date, "regime"]
    cov = scenario_covs[date]["stress" if regime == "high_vol" else "base"]

    Q = params["lambda_risk"] * risk_term(cov)

    # Linear penalties
    linear = (
        params["lambda_cost"] * linear_change_term(prev_sel) +
        params["lambda_turnover"] * linear_change_term(prev_sel)
    )

    # Cardinality
    Qc, lc = cardinality_penalty(len(prev_sel), params["cardinality_max"])
    Q += params["lambda_cardinality"] * Qc
    linear += params["lambda_cardinality"] * lc

    # Put linear terms on diagonal
    for i in range(len(prev_sel)):
        Q[i, i] += linear[i]

    return Q


In [23]:
# sanity check on one date 

test_date = dates[10]
Q = build_qubo(test_date, prev_selection, PARAMS)

Q.shape, np.allclose(Q, Q.T)


((25, 25), True)

In [24]:
# toy vaidation 5 assets 

Q_small = Q[:5, :5]
np.round(Q_small, 3)


array([[-95.199,   5.001,   5.001,   5.   ,   5.001],
       [  5.001, -95.199,   5.001,   5.   ,   5.001],
       [  5.001,   5.001, -95.198,   5.001,   5.001],
       [  5.   ,   5.   ,   5.001, -95.199,   5.   ],
       [  5.001,   5.001,   5.001,   5.   , -95.199]])